<a href="https://colab.research.google.com/github/DojunPark/Research_Undergraduate_Student_Program/blob/master/%5Bmachinetranslation%5D%2002_transformer%20modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Oct 16 05:08:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load the training data
- 200,000 pairs of parallel corpus

In [ ]:
pwd

'/content'

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/news_200k.csv')
df = df[['kor', 'eng']]

print(df.shape)

(200011, 2)


In [ ]:
df.sample(10)

,kor,eng
85813,"다음으로 KIA 타이거즈 김윤동, 롯데 구승민, NC 강윤구, SK 김태훈, 넥센 ...","Next, the KIA Tigers' Kim Yoon-dong, Lotte'S K..."
22953,이이경과 안소희가 세기의 거지를 가리기 위한 불꽃 신경전을 펼친다.,Lee Yi-kyung and Ahn So-hee will stage a flame...
77973,현실적으로 상급학교 진학 또는 사회 진출을 앞둔 학생들이 사실상 마지막 겨울방학 때...,"In reality, it can be interpreted that student..."
19538,범튀르크주의나 범이슬람주의를 의식해 역사를 자의적으로 해석한 것이란 비판도 나온다.,Critics say that the book is an arbitrary inte...
73595,법적 근로시간을 맞추려면 버스기사의 채용이 시급한데 낮은 임금 등으로 인해 버스기사...,While hiring bus drivers is urgent to meet leg...
169342,쏭류핑은 “설령 국방수권법이 헌법에 부합한다고 가정해도 미국의 국가안보를 진전시키거...,"""Even assuming that the National Defense Autho..."
43046,낙태를 선택할 수밖에 없는 특별한 사정을 고려한 특별법은 이로부터 20년이 지난 박...,A special law was created during the Park Chun...
107896,프라임칼리지의 가장 큰 장점으로는 수업에서 시험까지 모든 교육과정이 100% 온라인...,The main advantage of Prime College is that al...
91498,지난달 타이베이에 창설된 청현재이 캘리그라피 북부선교팀과 한국에서 오랜 기간 사역을...,It is the joint event held by Cheong-Hyeon-Jei...
184359,앞서 신 전 사무관은 지난해 12월 본인의 유튜브 계정을 통해 재직 당시 청와대가 ...,"Previously, a former Deputy Director Shin said..."


# Preprocessing the training data

- konlpy 라이브러리의 한국어 형태소분석기 Mecab 설치

In [ ]:
!pip install konlpy
! sudo apt-get install curl git
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 10.5MB/s 
     |████████████████████████████████| 3.8MB 48.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.10).
git is already the newest version (1:2.17.1-1ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Installing automake (A dependency for mecab-ko)
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRel

- spacy 라이브러리의 영어 형태소분석기 설치

In [ ]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
from konlpy.tag import Mecab
import spacy

mecab = Mecab()
spacy_en = spacy.load('en')

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

print('tokenization test with sample texts')
print('tokenizing for Korean: ', mecab.morphs('안녕하세요 저는 건국대학교에 재학 중인 박도준입니다.'))
print('tokenizing for English: ', tokenize_en('Hello, I am Dojun Park, a student at Konkuk University.'))

tokenization test with sample texts
tokenizing for Korean:  ['안녕', '하', '세요', '저', '는', '건국대', '학교', '에', '재학', '중', '인', '박도준', '입니다', '.']
tokenizing for English:  ['Hello', ',', 'I', 'am', 'Dojun', 'Park', ',', 'a', 'student', 'at', 'Konkuk', 'University', '.']


In [ ]:
from torchtext.data import Field, TabularDataset, BucketIterator

korean = Field(tokenize=mecab.morphs, lower=True, init_token='<sos>', eos_token='<eos>')
english = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>')

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


- training, test, vaildation data의 비율을 6:2:2로 분할

In [ ]:
from sklearn.model_selection import train_test_split

ko_train, ko_test, en_train, en_test = train_test_split(df['kor'], df['eng'], test_size=0.2, random_state=1)
ko_train, ko_val, en_train, en_val = train_test_split(ko_train, en_train, test_size=0.25, random_state=1)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=1)
train_data, valid_data = train_test_split(train_data, test_size=0.25, random_state=1)

In [ ]:
print('number of train_data: {}, {:.1f}%'.format(len(train_data), len(train_data)/(len(train_data)+len(test_data)+len(valid_data))))
print('number of test_data: {}, {:.1f}%'.format(len(test_data), len(test_data)/(len(train_data)+len(test_data)+len(valid_data))))
print('number of valid_data: {}, {:.1f}%'.format(len(valid_data), len(valid_data)/(len(train_data)+len(test_data)+len(valid_data))))

number of train_data: 120006, 0.6%
number of test_data: 40003, 0.2%
number of valid_data: 40002, 0.2%


- make vocab list

In [ ]:
korean.build_vocab(train_data['kor'], max_size=10000, min_freq=2)
english.build_vocab(train_data['eng'], max_size=10000, min_freq=2)

# Model designing

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter


class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_length,
        device):

        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)
        self.device = device
        self.Transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout)
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        # src shape: (src_len, N)
        src_mask = src.transpose(0, 1) == self.src_pad_idx
        # (N, src_len)
        return src_mask

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        src_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length).unsqueeze(1).expand(scr_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length).unsqueeze(1).expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )

        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequenct_mask(trg_seg_length).to(
            self.device)

        out = self.trnsformer(
            embed_src,
            embed_trg,
            src_key_padding_mask = src_padding_mask,
            tgt_mask = trg_mask
        )
        out = self.fc_out(out)

        return out

In [ ]:
# Setup the training phase
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
load_model = False
save_model = True

# Training hyperparameters
num_epochs = 5
learning_rate = 3e-4
batch_size = 32

# Model hyperparameters
src_vocab_size = len(korean.vocab)
trg_vocab_size = len(english.vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 3  # in the paper 6
num_decoder_layers = 3
dropout = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = english.vocab.stoi['<pad>']

# Tensorboard for nice plots
write = SummaryWriter('runs/loss_plot')
step = 0

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = batch_size,
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device
)

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
pad_idx = english.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
